<a href="https://colab.research.google.com/github/MaxsonCM/my_first_llm_google_gemini/blob/main/meu_proximo_anime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importa e defina a função para fazer requests

In [ ]:
import requests
import json
#doc api
#https://docs.api.jikan.moe/#tag/anime/operation/getAnimeForum
def pesquisa_anime(nome_anime, limite=0):

  url = "https://api.jikan.moe/v4/anime?q=" + nome_anime + "&limit=1"
  # Criar a requisição GET
  response = requests.get(url)
  filtered_data = []

  # Verificar se a requisição foi bem-sucedida
  if response.status_code == 200:
    # Converter o conteúdo da resposta para JSON
    dados_resposta = response.json()

    # Iterar pelos itens na chave 'data'
    for item in dados_resposta['data']:
        # Obter o título do tipo "Default"
        default_title = next((title['title'] for title in item['titles'] if title['type'] == "Default"), "N/A")

        # Coletar os gêneros e temas em listas
        genres = [genre['name'] for genre in item['genres']]
        themes = [theme['name'] for theme in item['themes']]

        # Criar um dicionário com as informações desejadas
        filtered_info = {
            'title': default_title,
            'synopsis': item['synopsis'],
            'genres': genres,
            'themes': themes
        }

        # Adicionar o dicionário filtrado à lista
        filtered_data.append(filtered_info)

  return filtered_data

def ultimos_animes():

  url = "https://api.jikan.moe/v4/anime?start_date=2024-01-01&end_date=2024-12-31&limit=25&sort=desc"

  # Criar a requisição GET
  response = requests.get(url)
  filtered_data = []

  # Verificar se a requisição foi bem-sucedida
  if response.status_code == 200:
    # Converter o conteúdo da resposta para JSON
    dados_resposta = response.json()
    # Iterar pelos itens na chave 'data'
    for item in dados_resposta['data']:
        # Obter o título do tipo "Default"
        default_title = next((title['title'] for title in item['titles'] if title['type'] == "Default"), "N/A")

        # Coletar os gêneros e temas em listas
        genres = [genre['name'] for genre in item['genres']]
        themes = [theme['name'] for theme in item['themes']]

        # Criar um dicionário com as informações desejadas
        filtered_info = {
            'title': default_title,
            'synopsis': item['synopsis'],
            'genres': genres,
            'themes': themes
        }

        # Adicionar o dicionário filtrado à lista
        filtered_data.append(filtered_info)

  return filtered_data


###Gera a lista de animes mais recentes


In [ ]:
lista_animes_novos = ultimos_animes()
print(lista_animes_novos)

[{'title': 'Chiyu Mahou no Machigatta Tsukaikata', 'synopsis': "Ken Usato, an ordinary high schooler, wishes for something fantastical to occur in his unremarkable life. Such an opportunity arrives when he is transported to another world alongside Kazuki Ryuusen and Suzune Inukami, two prodigious members of his school's student council. Arriving in Llinger Kingdom, the three are deemed to be the heroes tasked with stopping the impending invasion by the Demon Lord's army. However, this is a misunderstanding—Usato was summoned by accident and, unlike his two friends, is not one of the heroes.\n\nDespite this disheartening revelation, Usato learns that he has an aptitude for the extremely rare healing magic. This catches the attention of Rose, the intimidating captain of the kingdom's Rescue Team, who forcibly takes custody of Usato to mold him into a full-fledged healer. As he undergoes grueling training under Rose's supervision, Usato resolves to become capable enough to protect his fri

##Importa e configura o Google Gemini

In [ ]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

from google.colab import userdata
gemini_api = userdata.get('gemini_api')

#Configurações iniciais
import google.generativeai as genai

genai.configure(api_key=gemini_api)

generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}
safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

chat = model.start_chat(history=[])



##Reseta o prompt

In [ ]:
prompt = ""
#prompt_padrao = "Você se comportará como um algoritmo de sugestão de animes e deverá solicitar um nome de anime para depois pesquisar qual é sua sinopse e genero para sugerir outros animes similares"
prompt_padrao = "Retorne em linguagem natural apenas a sugestão de animes baseando no meu gosto de animes passado na estrutura json logo em seguida, analise o estilo do anime e sinopse para se basear em suas sugestões, passarei a lista de animes mais atual para serem selecionados"


##Cria um loop de perguntas

In [ ]:
while prompt != "fim":
  prompt = input('Deixe sugerir outros animes com base em um que você goste?\n')

  if prompt == "fim":
    break

  #recupera os dados do anime solicitado
  meu_anime = pesquisa_anime(prompt)
  if len(json.dumps(meu_anime)) == 0 or json.dumps(meu_anime)== "[]":
      print("Não encontrei esse anime, favor informar outro!")
      break
  else:

    response = chat.send_message(prompt_padrao + " Eu gosto do anime: " + json.dumps(meu_anime) + " Selecione animes com base nessa lista: " + json.dumps(lista_animes_novos))
    #response = chat.send_message(prompt_padrao + " Eu gosto do animes:" + json.dumps(meu_anime["data"]) + " Essas é a lista de animes mais atual:" + json.dumps(lista_animes_novos["data"]))
    print(response.text)

